In [13]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [44]:
### SETUP ###

import tweepy as tw
import pandas as pd

API_KEY = "6KmfpPDEF6ZHwtYdRzrDoDSn8"
API_SECRET = "NAm8MO1hTnz2debLFzNEpqkJcUZ2itwTCkXcod78t42pWrmzON"

class TwitterApi:
    def __init__(self):
        self.api = None
        self.raw_tweets = []
        self.clean_tweets = []
        
        self.authenticate_api()
        
    def exception_handler(func):
        def inner_function(*args, **kwargs):
            try:
                func(*args, **kwargs)
            except TypeError:
                print(f"{func.__name__} error")
        return inner_function
        
    @exception_handler
    def authenticate_api(self, api_key=API_KEY, api_secret=API_SECRET):
        auth = tw.OAuthHandler(API_KEY, API_SECRET)
        # authenticate
        self.api = tw.API(auth, wait_on_rate_limit=True)
    
    @exception_handler
    def check_rate_limit(self):
        data = self.api.rate_limit_status()
        return data
    
    @staticmethod
    def limit_handled(cursor):
        while True:
            try:
                yield cursor.next()
            except tw.RateLimitError as e:
                raise(e)
                time.sleep(15 * 60)

    def save_data(self, tweets_list, method):
        if method == 'append':
            self.raw_tweets.append(tweets_list)
        else:
            self.raw_tweets = tweets_list
        
    def get_raw_tweets_from_api(
        self, 
        q, 
        since, 
        until=None, 
        lang="en", 
        items=500, 
        method='append'
    ):
        tweets_list = []
        try:
            for tweet in self.limit_handled(
                tw.Cursor(self.api.search,
                      q=q,
                      lang=lang,
                      since=since, 
                      until=until).items(items)
            ):
                tweets_list.append(tweet._json)
        except RuntimeError as e:
            raise(e)   
        
        if len(tweets_list):
            self.save_data(tweets_list, method)
        else:
            print('get_raw_tweets_from_api, no tweets!')

    
    def get_clean_tweets(self):
        self.clean_tweets_df = pd.json_normalize(self.raw_tweets)
    
    def get_twitter_data_as_dataframe(
        self, 
        q, 
        since, 
        until=None, 
        lang="en", 
        items=500, 
        method='append'
    ):
        self.get_raw_tweets_from_api(q, since, until, lang, items, method)
        self.get_clean_tweets()
        return self.clean_tweets_df
    

In [45]:
SEARCH_QUERY = "#globalwarming -filter:retweets"
## keep these within a year or we will break twitter
SINCE = "2021-01-01"
UNTIL = "2021-01-31"
METHOD = 'overwrite'
LANG = "en"
ITEMS = 500

api = TwitterApi()
rate_limit = api.check_rate_limit()
print(rate_limit)
tweet_df = api.get_twitter_data_as_dataframe(q=SEARCH_QUERY, since=SINCE, until=UNTIL, lang=LANG, method=METHOD, items=ITEMS)


TweepError: Twitter error response: status code = 401

In [33]:
tweet_df

""


In [ ]:
### TWEET BODY
import json
first_tweet = tweets_copy[0]
tweet_fields = first_tweet._json.keys()
for tweet_field in tweet_fields:
    print(tweet_field)

In [ ]:
# TWEET USER BODY
tweet_fields_user = first_tweet.user._json.keys()
for tweet_field in tweet_fields_user:
    print(tweet_field)